<a href="https://colab.research.google.com/github/Saw20211/20238436_Zork_CS4076/blob/main/Lab_Week10_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EV EVALUATION



#### EV- 0.0 Test sentence similarity models (optional)
Just to play with a couple of models and a few sentences.

In [1]:
# Install Transformers
from IPython.display import clear_output
! pip install -U sentence-transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125924 sha256=bfa0e98d707688e36cf692e1f655f480d22d4b9aec5e9bfe76dade5dea4d268b
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1


In [2]:
from sentence_transformers import SentenceTransformer, util
# model = SentenceTransformer('all-MiniLM-L6-v2')
model = SentenceTransformer('nli-distilroberta-base-v2')

clear_output()

# Two lists of sentences
sentences1 = ['The cat sits outside',
             'A man is playing guitar',
             'The new movie is awesome',
              # Triples
              # 'Aarhus_Airport | cityServed | Aarhus,_Denmark',
              # 'Aarhus_Airport | location | Tirstrup',
              # "Textified" triples
              # 'Aarhus Airport city served Aarhus, Denmark',
              # 'Aarhus Airport location Tirstrup'
              ]

sentences2 = ['The dog plays in the garden',
              'A woman watches TV',
              'The new movie is so great',
              # 'Aarhus Airport serves the city of Aarhus, Denmark',
              # 'Aarhus Airport is in Tirstrup',
              # 'Aarhus Airport serves the city of Aarhus, Denmark',
              # 'Aarhus Airport is in Tirstrup'
              ]

#Compute embedding for both lists
embeddings1 = model.encode(sentences1, convert_to_tensor=True)
embeddings2 = model.encode(sentences2, convert_to_tensor=True)

#Compute cosine-similarities
cosine_scores = util.cos_sim(embeddings1, embeddings2)

#Output the pairs with their score
for i in range(len(sentences1)):
    print("{} \t\t {} \t\t Score: {:.4f}".format(sentences1[i], sentences2[i], cosine_scores[i][i]))

The cat sits outside 		 The dog plays in the garden 		 Score: 0.2289
A man is playing guitar 		 A woman watches TV 		 Score: -0.0171
The new movie is awesome 		 The new movie is so great 		 Score: 0.9504


##**EV-1 Setup**

#### EV-1.1 Mount drive (optional)

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#### EV-1.2 Download files

In [7]:
# ! pip install --upgrade --no-cache-dir gdown
from IPython.display import clear_output
! gdown 1M2sxOCSUQJiLt6yC40fGI0QWSeq7jVod
! unzip '/content/Lab_Week10.zip'
clear_output()

#### EV-1.3 Install transformers

In [8]:
from IPython.display import clear_output
!pip install -U sentence-transformers==2.2.2
clear_output()

##**EV-2 Load files and model**  (INPUT NEEDED: path to fine-tuned model)

In [9]:
#@title Path of fine-tuned model to evaluate
fineTunedTransformer_path = '/content/drive/MyDrive/Colab-dump/Lab_Week10/MyModel-nli-distilroberta-base-v2-2023-11-10_15-21-23'#@param string

####EV-2.1 Load models, set paths

In [10]:
from sentence_transformers import SentenceTransformer, util
import os

files_path = '/content/Lab_Week10/Eval/'
eval_scores_candidates = '/content/Lab_Week10/Eval/results_candSent/'

# fineTunedTransformer_path = '/content/drive/MyDrive/Colab-dump/Lab_Week10/MyModel-nli-distilroberta-base-v2-2023-03-22_12-17-11'
# fineTunedTransformer_path = '/content/drive/MyDrive/Colab-dump/MyModel-nli-distilroberta-base-v2-2023-03-16_12-47-24_W2020_noTypes-noTags_TrueCasedText'
# fineTuned_model = SentenceTransformer(fineTunedTransformer_path)
fineTuned_model = None
if os.path.exists(fineTunedTransformer_path):
  fineTuned_model = SentenceTransformer(fineTunedTransformer_path)
offTheShelf_model = SentenceTransformer('nli-distilroberta-base-v2')

clear_output()

####EV-2.2 Load files

In [11]:
import pickle
from tqdm import tqdm

with open(files_path + 'candidateSentences_dev_1.txt', 'rb') as f:
  allSentences = list(pickle.load(f))

with open(files_path + 'targetSentences_dev_1.txt', 'rb') as f:
  referenceSentences = list(pickle.load(f))

with open(files_path + 'triples_dev_1_textified.txt', 'rb') as f:
  textifiedTriples = pickle.load(f)

In [12]:
# print('Expected: 1834, 401, 401')
print('Expected: 961, 401, 401')
print(len(allSentences), len(textifiedTriples), len(referenceSentences))
print(referenceSentences[1], textifiedTriples[1])

Expected: 961, 401, 401
961 401 401
["Aarhus Airport's runway length is 2702.0.", 'The Aarhus Airport has a runway length of 2702.0.'] Aarhus Airport runway length 2702.0 .


## **EV-3 Scoring of candidate sentences (use GPU)**
Run for fine-tuned model only.

####EV-3.1 Scoring Functions

In [13]:
from tqdm import tqdm

def evaluate(textified_triple, sentences_list, model):
    to_embed = [[textified_triple], sentences_list]
    to_embed = [element for sublist in to_embed for element in sublist]

    embeddings = model.encode(to_embed, convert_to_tensor=True)

    results = []

    for i in range(1, len(embeddings)):
        similarity = float(util.pytorch_cos_sim(embeddings[0], embeddings[i])[0][0])
        results.append([sentences_list[i-1], similarity])

    results.sort(key=lambda result: result[1], reverse=True)
    return results

def evaluate_allTriples(textifiedTriples, sentences_list, savePath, model, ft=False):
    for i, textified_triple in enumerate(tqdm(textifiedTriples)):
        results = evaluate(textified_triple, sentences_list, model)
        if ft == False:
            with open(savePath + 'triple'+str(i)+'_results2.txt', 'wb') as fh:
                pickle.dump(results, fh)
        else:
            with open(savePath + 'triple'+str(i)+'_results1.txt', 'wb') as fh:
                pickle.dump(results, fh)

def evaluate_allTriples2(textifiedTriples, referenceSentences, savePath, model, ft=False):
    for i, textified_triple in enumerate(tqdm(textifiedTriples)):
        results = evaluate(textified_triple, list(referenceSentences[i]), model)
        if ft == False:
            with open(savePath + 'triple'+str(i)+'_results2.txt', 'wb') as fh:
                pickle.dump(results, fh)
        else:
            with open(savePath + 'triple'+str(i)+'_results1.txt', 'wb') as fh:
                pickle.dump(results, fh)

####EV-3.2 Run scoring

In [14]:
# The results for the off-the-shelf module are already provided, no need to run this cell
# evaluate_allTriples(textifiedTriples, allSentences, eval_scores_candidates, offTheShelf_model)

In [15]:
evaluate_allTriples(textifiedTriples, allSentences, eval_scores_candidates, fineTuned_model, ft=True)

  0%|          | 0/401 [00:00<?, ?it/s]


AttributeError: 'NoneType' object has no attribute 'encode'

#### EV-3.3 Print raw results

In [18]:
def print_result(textified_triple, results, n=None):
    print('---', textified_triple, '---')
    if not n or n > len(results):
        n = len(results)
    for i in range(n):
   # for i, result in enumerate(results):
        print(str(i+1)+'.', 'Similarity: {:.3f}'.format(results[i][1]), '->', results[i][0])

def printResults(files_path, model, n):

    results_path = eval_scores_candidates
    with open (files_path + 'triples_dev_1_textified.txt', 'rb') as f:
        textifiedTriples = pickle.load(f)

    for i in range(n):

        textified_triple = textifiedTriples[i]

        if model == 'offTheShelf':
            result2_name = 'triple'+str(i)+'_results2.txt'
            pickle_off = open(results_path + result2_name, 'rb')
            result2 = pickle.load(pickle_off)
            print('OffTheShelf Model')
            print_result(textified_triple, result2, n=10)
            print()
            print()

        elif model =='fineTuned':
            result1_name = 'triple'+str(i)+'_results1.txt'
            pickle_off = open(results_path + result1_name, 'rb')
            result1 = pickle.load(pickle_off)
            print('FineTuned Model')
            print_result(textified_triple, result1, n=10)
            print()
            print()

        elif model =='both':
            result1_name = 'triple'+str(i)+'_results1.txt'
            result2_name = 'triple'+str(i)+'_results2.txt'
            pickle_off = open(results_path + result1_name, 'rb')
            result1 = pickle.load(pickle_off)
            pickle_off = open(results_path + result2_name, 'rb')
            result2 = pickle.load(pickle_off)
            print('FineTuned Model')
            print_result(textified_triple, result1, n=10)
            print()
            print('OffTheShelf Model')
            print_result(textified_triple, result2, n=10)
            print()
            print()

In [19]:
# Second parameter can be 'offTheShelf', 'fineTuned' or 'both'
model = 'offTheShelf'#@param ['offTheShelf', 'fineTuned', 'both']
# Third parameter is the number of outputs to print (up to 401)
printResults(files_path, 'offTheShelf', 2)

OffTheShelf Model
--- Aarhus leader Jacob Bundsgaard . ---
1. Similarity: 0.960 -> The leader of Aarhus is Jacob Bundsgaard.
2. Similarity: 0.691 -> Lars Løkke Rasmussen is the leader of Denmark.
3. Similarity: 0.660 -> Lars Løkke Rasmussen leads Denmark.
4. Similarity: 0.642 -> Denmark's leader name is Lars Løkke Rasmussen.
5. Similarity: 0.561 -> The Aarhus University School of Business and Social Sciences is in the country of Denmark.
6. Similarity: 0.550 -> The School of Business and Social Sciences at the Aarhus University is in Denmark.
7. Similarity: 0.543 -> The School of Business and Social Sciences at the Aarhus University is situated in Denmark.
8. Similarity: 0.540 -> The School of Business and Social Sciences at the Aarhus University is located in Denmark.
9. Similarity: 0.532 -> Copenhagen is the capital of Denmark.
10. Similarity: 0.480 -> The capital of Denmark is Copenhagen.


OffTheShelf Model
--- Aarhus Airport runway length 2702.0 . ---
1. Similarity: 0.967 -> Aarhu

## **EV-4 Evaluation results aggregation** (INPUT NEEDED: model to evaluate)
Takes the files produced in the scoring phase and extracts global results of the model.

#### EV-4.1 Functions and pre-processing

In [55]:
import pickle
from tqdm import tqdm
import statistics

# To define thresholds to be tested
thresholds = []
threshold = 0
for i in range(99):
    threshold += 0.01
    thresholds.append(round(threshold, 2))


def classify_results(results, threshold, correctSentences, chooseTest):
    truePositives = 0
    trueNegatives = 0
    falsePositives = 0
    falseNegatives = 0
    for result in results:
        if chooseTest == 'originalTriples':
            if result[1] >= threshold:
                if result[0] in correctSentences:
                    truePositives += 1
                else:
                    falsePositives += 1
            else:
                if result[0] in correctSentences:
                    falseNegatives += 1
                else:
                    trueNegatives += 1
        else:
            if result[1] >= threshold:
                falsePositives += 1
            else:
                trueNegatives += 1

    classifiedResults = [truePositives, falsePositives, trueNegatives, falseNegatives]

    return classifiedResults

def top_isCorrect(results, correctSentences):
    top_isCorrect = True
    top = [result[0] for result in results[:len(correctSentences)]]
    for correctSentence in correctSentences:
        if correctSentence not in top:
            top_isCorrect = False
            break
    return top_isCorrect

def get_correctSentencesMean(results, correctSentences, topIsCorrect):
    if topIsCorrect:
        correctSentencesMean = statistics.mean([result[1] for result in results[:len(correctSentences)]])
    else:
        correctSentencesScores = []
        for correctSentence in correctSentences:
            for result in results:
                if result[0] == correctSentence:
                    correctSentencesScores.append(result[1])
        correctSentencesMean = statistics.mean(correctSentencesScores)

    return correctSentencesMean

def get_classificationMetrics(classifiedResults_sum, threshold_index):
    tp = classifiedResults_sum[threshold_index][0]
    fp = classifiedResults_sum[threshold_index][1]
    tn = classifiedResults_sum[threshold_index][2]
    fn = classifiedResults_sum[threshold_index][3]

    if tp == 0:
        precision = 0
        recall = 0
        f1 = 0
    else:
        precision = tp / (fp + tp)
        recall = tp / (fn + tp)
        f1 = (2 * precision * recall) / (precision + recall)

    accuracy = (fp + tn) / (tp + fn + tn + fp)

    metrics = [precision, recall, accuracy, f1]

    return metrics

def get_allClassificationMetrics(thresholds, classifiedResults_sum):

    allMetrics = []

    for i, threshold in enumerate(thresholds):
        metrics = get_classificationMetrics(classifiedResults_sum, i)
        allMetrics.append(metrics)

    return allMetrics

def get_allProperties(triples):
    all_properties = set()
    for triple in triples:
        all_properties.add(triple[1])
    return all_properties

In [56]:
# with open (files_path + 'triples_dev_1_textified.txt', 'rb') as f:
#     textifiedTriples = pickle.load(f)

# notParenthesesTriples = []
# for triple in textifiedTriples:
#     if not '(' in triple:
#         notParenthesesTriples.append(triple)
# print('Expected: 350, 401')
# print(len(notParenthesesTriples), len(textifiedTriples))

# with open (files_path + 'targetSentences_dev_1.txt', 'rb') as f:
#     sentences = pickle.load(f)
# print(textifiedTriples[111], sentences[111])

In [57]:
originalTriples_path2 = ''

def checkResults(thresholds, n, files_path, chooseTest, model, subObjSentences=False):

    if chooseTest == 'originalTriples':
        if subObjSentences:
            results_path = originalTriples_path2
        else:
            results_path = eval_scores_candidates
        with open (files_path + 'triples_dev_1_textified.txt', 'rb') as f:
            textifiedTriples = pickle.load(f)
        with open (files_path + 'targetSentences_dev_1.txt', 'rb') as f:
            sentences = pickle.load(f)
        '''
        notParenthesesIndexes = []
        for i, triple in enumerate(textifiedTriples):
            if not '(' in triple:
                notParenthesesIndexes.append(i)
        '''
    else:
        if chooseTest == 'exchangedObjSubTriples':
            results_path = exchangedObjSubTriplesResults_path
            with open (files_path + 'textified_exchangedObjSubTriples.txt', 'rb') as f:
                textifiedTriples = pickle.load(f)

        elif chooseTest == 'randomPropertyTriples':
            results_path = randomPropertyTriplesResults_path
            with open (files_path + 'textified_randomPropertyTriples.txt', 'rb') as f:
                textifiedTriples = pickle.load(f)

        else:
            raise ValueError("chooseTest must be one of the following: 'originalTriples', 'exchangedObjSubTriples', 'randomPropertyTriples'")


    with open (files_path + 'triples_dev_1_split.txt', 'rb') as f:
        triples = pickle.load(f)
    properties_set = get_allProperties(triples)
    triples_properties = [triple[1] for triple in triples]
    properties_errors = {property: 0 for property in properties_set}

    '''with open (files_path + 'triples_categories.txt', 'rb') as f:
        triples_categories = pickle.load(f)'''

    highestScores = []
    # offTheShelf_highestScores = []

    '''categories_set = set(triples_categories)
    categories_errors = {category: 0 for category in categories_set}'''

    classifiedResults_sum = [[0]*4 for i in range(len(thresholds))]   #[truePositives, trueNegatives, falsePositives, falseNegatives] for each threshold
    incorrectTops_sum = 0
    incorrectTop1_sum = 0
    correctSentences_means = []
    differences = []

    fineTuned_errors = []

    #for i in tqdm(notParenthesesIndexes):
    for i, textifiedTriple in enumerate(tqdm(textifiedTriples)):
    #for i in tqdm(range(textifiedTriples)):

        #textifiedTriple = textifiedTriples[i]
        correctSentences = sentences[i] if chooseTest == 'originalTriples' else None

        results1 = ''
        results2 = ''
        if model == 'fineTuned':
            results1_name = 'triple'+str(i)+'_results1.txt'
            pickle_off = open(results_path + results1_name, 'rb')
            results1 = pickle.load(pickle_off)
            highestScores.append(results1[0][1])
            for j, threshold in enumerate(thresholds):
                fineTuned_classifiedResults = classify_results(results1, threshold, correctSentences, chooseTest)
                classifiedResults_sum[j] = [i+j for i,j in zip(classifiedResults_sum[j], fineTuned_classifiedResults)]

        elif model == 'offTheShelf':
            results2_name = 'triple'+str(i)+'_results2.txt'
            pickle_off = open(results_path + results2_name, 'rb')
            results2 = pickle.load(pickle_off)
            highestScores.append(results2[0][1])
            for j, threshold in enumerate(thresholds):
                offTheShelf_classifiedResults = classify_results(results2, threshold, correctSentences, chooseTest)
                classifiedResults_sum[j] = [i+j for i,j in zip(classifiedResults_sum[j], offTheShelf_classifiedResults)]


        metrics = get_allClassificationMetrics(thresholds, classifiedResults_sum)

        model_f1List = [[threshold, metrics[i][3]] for i, threshold in enumerate(thresholds)]
        model_f1Top = sorted(model_f1List, key=lambda x: x[1], reverse=True)
        # offTheShelf_f1List = [[threshold, offTheShelf_metrics[i][3]] for i, threshold in enumerate(thresholds)]
        # offTheShelf_f1Top = sorted(offTheShelf_f1List, key=lambda x: x[1], reverse=True)

        if chooseTest == 'originalTriples':
            topIsCorrect = ''
            if model == 'fineTuned':
                topIsCorrect = top_isCorrect(results1, correctSentences)
                correctSentencesMean = get_correctSentencesMean(results1, correctSentences, topIsCorrect)
                if results1[0][0] not in correctSentences:
                   incorrectTop1_sum += 1
            elif model == 'offTheShelf':
                topIsCorrect = top_isCorrect(results2, correctSentences)
                correctSentencesMean = get_correctSentencesMean(results2, correctSentences, topIsCorrect)
                if results2[0][0] not in correctSentences:
                    incorrectTop1_sum += 1

            correctSentences_means.append(correctSentencesMean)

            difference = ''

            if topIsCorrect:
                if model == 'fineTuned':
                    if len(correctSentences) < len(results1):
                        difference = correctSentencesMean - results1[len(correctSentences)][1]
                    else:
                        difference = 0.212 ##
                elif model == 'offTheShelf':
                    if len(correctSentences) < len(results2):
                        difference = correctSentencesMean - results2[len(correctSentences)][1]
                    else:
                        difference = 0.110
            else:
                incorrectTops_sum += 1
                difference = None
                if model == 'fineTuned':
                   fineTuned_errors.append([textifiedTriple, correctSentences, results1[:5]])
                '''categories_errors[triples_categories[i]] += 1'''
                properties_errors[triples_properties[i]] += 1

            if model == 'fineTuned':
                correctSentences_means.append(get_correctSentencesMean(results1, correctSentences, topIsCorrect))
            elif model == 'offTheShelf':
                correctSentences_means.append(get_correctSentencesMean(results2, correctSentences, topIsCorrect))

            differences.append(difference)

    highestScores_mean = statistics.mean(highestScores)

    if chooseTest == 'originalTriples':
        '''category_errors = [[category, categories_errors[category]] for category in categories_errors]'''
        property_errors = [[property, properties_errors[property]] for property in properties_errors]

        incorrectTops_percentage = incorrectTops_sum / n * 100

        incorrectTop1_percentage = incorrectTop1_sum / n * 100

        correctSentencesMeans_mean = statistics.mean(correctSentences_means)

        differences_mean = statistics.mean([difference for difference in filter(None, differences)])

        results = [metrics,
                model_f1Top,
                classifiedResults_sum,
                incorrectTops_percentage,
                incorrectTop1_percentage,
                correctSentencesMeans_mean,
                differences_mean,
                highestScores_mean,
                fineTuned_errors, thresholds, property_errors]
    else:
        results = [metrics,
                model_f1Top,
                classifiedResults_sum,
                highestScores_mean,
                thresholds]

    return results

def print_infoResults(results, chooseTest, model):
    if chooseTest == 'originalTriples':
        [metrics,
        model_f1Top,
        classifiedResults_sum,
        incorrectTops_percentage,
        incorrectTop1_percentage,
        correctSentencesMeans_mean,
        differences_mean,
        highestScores_mean,
        fineTuned_errors, thresholds, property_errors] = results
    else:
         [metrics,
          model_f1Top,
          classifiedResults_sum,
          highestScores_mean,
          thresholds] = results

    print('CLASSIFICATION METRICS')
    for i, threshold in enumerate(thresholds):
        print(threshold)
        print('\tEvaluated model:  ')
        print('\t    [True Positives, False Positives]: [{:.0f}, {:.0f}]'.format(classifiedResults_sum[i][0], classifiedResults_sum[i][1]))
        print('\t    [False Negatives, True Negatives]: [{:.0f}, {:.0f}]'.format(classifiedResults_sum[i][3], classifiedResults_sum[i][2]))
        print('\t\tPrecision: {:.3f}'.format(metrics[i][0]))
        print('\t\tRecall: {:.3f}'.format(metrics[i][1]))
        print('\t\taccuracy: {:.3f}'.format(metrics[i][2]))
        print('\t\tf1-score: {:.3f}'.format(metrics[i][3]))
        print()
    print('TOP F1 THRESHOLDS')
    print('\tEvaluated model:')
    print('\t\t', model_f1Top[:5])
    print()
    print('HIGHEST SCORES MEAN')
    print('Evaluated model:  {:.5f}'.format(highestScores_mean))
    print()

    if chooseTest == 'originalTriples':
        print('NOT CORRECT TOPS')
        print('Evaluated model:  {:.2f}%'.format(incorrectTops_percentage))
        print()
        print('NOT CORRECT TOP1')
        print('Evaluated model:  {:.2f}%'.format(incorrectTop1_percentage))
        print()
        print('CORRECT SENTENCES SCORE MEANS MEAN')
        print('Evaluated model: ', '{:.3f}'.format(correctSentencesMeans_mean))
        print()
        print('DIFFERENCE BETWEEN TOP SCORE MEANS AND FIRST NON CORRECT MEAN')
        print('Evaluated model: ', '{:.3f}'.format(differences_mean))
        print()
        if model == 'fineTuned':
            print('FINE-TUNED MODEL ERRORS')
            for i, [textified_triple, correct_sentences, results] in enumerate(fineTuned_errors):
                print(i+1, '---', textified_triple) #, '---  (category: '+category+')')
                print('\tCorrect Sentences')
                for j, sentence in enumerate(correct_sentences):
                    print('\t\t', j+1, sentence)
                print('\tTop sentences')
                for j, result in enumerate(results):
                    print('\t\t', j+1, result[0], '=>', result[1])
                print()
            print()
        '''print('CATEGORY ERRORS')
        for [category, sum] in category_errors:
            print('\t', category+':', sum)
        print()'''
        # print('PROPERTY ERRORS')
        # for [property, sum] in property_errors:
        #     print('\t', property+':', sum)
        # print()

####EV-4.2 Run evaluation (pick model first)

In [58]:
#@title Pick one model type to evaluate
model = 'offTheShelf'#@param ['offTheShelf', 'fineTuned']
#'originalTriples', 'exchangedObjSubTriples', 'randomPropertyTriples'
results = checkResults(thresholds, 401, files_path, 'originalTriples', model)
print_infoResults(results, 'originalTriples', model)

100%|██████████| 401/401 [00:08<00:00, 49.06it/s]


CLASSIFICATION METRICS
0.01
	Evaluated model:  
	    [True Positives, False Positives]: [963, 366301]
	    [False Negatives, True Negatives]: [0, 18097]
		Precision: 0.003
		Recall: 1.000
		accuracy: 0.998
		f1-score: 0.005

0.02
	Evaluated model:  
	    [True Positives, False Positives]: [963, 361037]
	    [False Negatives, True Negatives]: [0, 23361]
		Precision: 0.003
		Recall: 1.000
		accuracy: 0.998
		f1-score: 0.005

0.03
	Evaluated model:  
	    [True Positives, False Positives]: [963, 354877]
	    [False Negatives, True Negatives]: [0, 29521]
		Precision: 0.003
		Recall: 1.000
		accuracy: 0.998
		f1-score: 0.005

0.04
	Evaluated model:  
	    [True Positives, False Positives]: [963, 347533]
	    [False Negatives, True Negatives]: [0, 36865]
		Precision: 0.003
		Recall: 1.000
		accuracy: 0.998
		f1-score: 0.006

0.05
	Evaluated model:  
	    [True Positives, False Positives]: [963, 338829]
	    [False Negatives, True Negatives]: [0, 45569]
		Precision: 0.003
		Recall: 1.000
		ac

#FT FINE TUNING

In [24]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## **FT-1 Dataset**

### FT-1.1 Load WebNLG v3.0

In [59]:
# For clearing outputs of installs
from IPython.display import clear_output

# datasets is for loading datasets from HuggingFace
!pip install datasets
from datasets import load_dataset

clear_output()

webnlg = load_dataset('web_nlg', 'release_v3.0_en', trust_remote_code=True)

Load triples of size 1 from the WebNLG training data

In [60]:
def load_webnlg(webnlg):
    dataset = []
    for sample in webnlg['train']:
        if sample['size'] == 1:
            dataset.append([sample['modified_triple_sets']['mtriple_set'][0][0], sample['lex']['text']])
    return dataset

dataset = load_webnlg(webnlg)

Merge repeated triples

In [61]:
def merge_repeated_triples(dataset):

    triples = [] #create a list of triples to then find the repeated ones
    for sample in dataset:
        triples.append(sample[0])

    duplicates = [triple for triple in triples if triples.count(triple) > 1] #duplicated triples list

    to_delete = []
    for i, sample in enumerate(dataset): #Append sentences from repeated triples in the first occurrence triple list
        triple = sample[0]
        if triple in duplicates:
            first_occurrence = triples.index(triple)
            if i != first_occurrence:
                for sentence in sample[1]:
                    dataset[first_occurrence][1].append(sentence)
                to_delete.append(i) #Save indexes of triples to delete

    for i, index in enumerate(to_delete): #Delete repeated triples
        del dataset[index - i]

    for sample in dataset: #Remove repeated sentences after merging
        sentence_set = set(sample[1])
        # Sort so that sentences are always in the same order (important for getting the same sampling given a random seed later on)
        sample[1] = sorted(list(sentence_set))

    return dataset

dataset = merge_repeated_triples(dataset)

In [62]:
print(dataset[0])
print(len(dataset))

['Aarhus_Airport | cityServed | "Aarhus, Denmark"', ['Aarhus Airport serves the city of Aarhus, Denmark.', 'The Aarhus is the airport of Aarhus, Denmark.']]
3107


Text triples: Each triple is a string in this form: "Subject | Property | Object"

In [63]:
def get_text_triples(dataset):
    text_triples = []
    for sample in dataset:
        text_triples.append(sample[0])
    return text_triples

text_triples = get_text_triples(dataset)

In [64]:
print('Num text triples:', len(text_triples), '(Expected: 3107)')
print()
print('text_triples[0]:')
text_triples[0]

Num text triples: 3107 (Expected: 3107)

text_triples[0]:


'Aarhus_Airport | cityServed | "Aarhus, Denmark"'

Sentences list: a list with all sentences corresponding to 1-triple inputs

In [65]:
def get_sentences(dataset):
    sentences = []
    for sample in dataset:
        for sentence in sample[1]:
            sentences.append(sentence)
    return sentences

sentences = get_sentences(dataset)

In [66]:
print('Num sentences:', len(sentences), '(Expected: 7630)')
print()
print('sentences[0]:')
sentences[0]

Num sentences: 7630 (Expected: 7630)

sentences[0]:


'Aarhus Airport serves the city of Aarhus, Denmark.'

Triples list: triples are stored as a list of 3 elements to access Subj, Property and Obj separately

In [67]:
def get_triples(text_triples):
    triples = []
    for text_triple in text_triples:
        firstBarIndex = text_triple.find('|')-1
        secondBarIndex = text_triple.rfind('|')+2

        sub = text_triple[:firstBarIndex]
        prop = text_triple[firstBarIndex + 3 : secondBarIndex-3]
        obj = text_triple[secondBarIndex:]

        triple = [sub, prop, obj]
        triples.append(triple)
    return triples

triples = get_triples(text_triples)

In [68]:
print('Num triples:', len(triples), '(Expected: 3107)')
print()
print('triples[0]:')
triples[0]

Num triples: 3107 (Expected: 3107)

triples[0]:


['Aarhus_Airport', 'cityServed', '"Aarhus, Denmark"']

In [69]:
# Save data
# import pickle

# fineTuningDataset_path = '/content/' #Write path to save the dataset
# with open(fineTuningDataset_path + 'triples_split.txt', 'wb') as fh:
#    pickle.dump(triples, fh)

# with open(fineTuningDataset_path + 'triples_raw.txt', 'wb') as fh:
#    pickle.dump(text_triples, fh)

### FT-1.2 Extract info needed for Finetuning dataset: CELLS TO EDIT in experiments!

Construct intermediate dataset: eventually, we want a dataset of sentence-like pairs with approximate similarity score between them.

In [70]:
# CELL TO EDIT!
# build a list of 2 dictionaries to store sentences with 2 different levels of similarity with the input triple
# if you want to create more splits, add dict() to intermediate_dataset_temp

############################################################# EDIT BELOW #############################################################
intermediate_dataset_temp = [dict(), dict()]
############################################################# EDIT ABOVE #############################################################
for category in intermediate_dataset_temp:
    for text_triple in text_triples:
        category[text_triple] = set()

In [71]:
print(len(intermediate_dataset_temp[0]))
print(intermediate_dataset_temp[0]['Aarhus_Airport | cityServed | "Aarhus, Denmark"'])

3107
set()


In [72]:
# CELL TO EDIT!
# For each triple, compare it to all other triples and get a set with the intersection of the two triples.
for i, triple1 in enumerate(triples):
    triple_1 = set(triple1)
    for j, triple2 in enumerate(triples):
        triple_2 = set(triple2)
        final = triple_1 & triple_2

        # Group together the sentences that verbalise triples that have all or no elements (Subj, Prop, Obj) in common with triple1
        # Keep the subsets ordered so we can use the order later on to assigne scores
        # E.g. the data with NO overlap should be in the first position of the intermediate_dataset_temp list, and the data with full overlap should be in the last position.
        ############################################################# EDIT BELOW #############################################################
        if len(final) == 0:
            for sentence in dataset[j][1]:
                intermediate_dataset_temp[0][text_triples[i]].add(sentence)
        elif len(final) == 3:
            for sentence in dataset[j][1]:
                intermediate_dataset_temp[1][text_triples[i]].add(sentence)
        ############################################################# EDIT ABOVE #############################################################
    # print(final)

intermediate_dataset = []

def listify_dicoValues(old_main_dico, category):
    """ Transforms a set into a list"""
    new_embedded_dico = {}
    # dico[category] is a dictionary too
    for input_triple in old_main_dico[category]:
        # convert set into sorted list
        sorted_list_sentences = sorted(list(old_main_dico[category][input_triple]))
        #create a new data point in the intermediate dataset with for each triple of a given category a list of sentences
        new_embedded_dico[input_triple] = sorted_list_sentences
    return(new_embedded_dico)

count = 0
while count < len(intermediate_dataset_temp):
    intermediate_dataset.append(listify_dicoValues(intermediate_dataset_temp, count))
    count += 1

In [73]:
# print(len(intermediate_dataset_temp[0]))
# print(list(intermediate_dataset_temp[0]['Aarhus_Airport | cityServed | "Aarhus, Denmark"'])[:10])
# print(list(intermediate_dataset_temp[1]['Aarhus_Airport | cityServed | "Aarhus, Denmark"'])[:10])
print(len(intermediate_dataset[0]))
print(list(intermediate_dataset[0]['Aarhus_Airport | cityServed | "Aarhus, Denmark"'])[:10])
print(list(intermediate_dataset[1]['Aarhus_Airport | cityServed | "Aarhus, Denmark"'])[:10])

3107
['" A Wizard of Mars was Published in the United States ".', '"1928 SJ" is the former name of 1101 Clematis.', '"1999 SN5" was the former name of (66063) 1998 RO1.', '"87000823" is Asilomar Conference Grounds\' reference number in the National Register of Historic Places.', '"AIP Adv." is the abbreviation of AIP Advances.', '"Abh.Math.Semin.Univ.Hambg" is the abbreviation for Abhandlungen aus dem Mathematischen Seminar der Universität Hamburg.', '"Drake Stevens" is known as the comic character Airman.', '"Maka-Diyos, Maka-Tao, Makakalikasan at Makabansa" is a motto of the Philippines.', '"Mark Desmond" is also known as the comic character Blockbuster.', '"Mayor" is the title of the leader in Atlantic City, New Jersey.']
['Aarhus Airport serves the city of Aarhus, Denmark.', 'The Aarhus is the airport of Aarhus, Denmark.']


Save dataset

In [74]:
import pickle

fineTuningDataset_path = '/content/' #Write path to save the dataset
with open(fineTuningDataset_path + 'intermediate_dataset.txt', 'wb') as fh:
   pickle.dump(intermediate_dataset, fh)

### FT-1.3 Create text/string versions of the triples to be compared to the sentences

In [75]:
# print(triples[0])

def camelCaseClean(text):
  words = [[text[0]]]
  for c in text[1:]:
    if words[-1][-1].islower() and c.isupper():
      words.append(list(c.lower()))
    else:
      words[-1].append(c)
  words = [''.join(word) for word in words]
  cleaned = ' '.join(words)
  return cleaned

textified_triples_noTypes_noTags = []

for triple in triples:
  # remove underscores and quotes
  subj = triple[0].replace('_',' ').replace('"','')
  obj = triple[2].replace('_',' ').replace('"','')
  # split property names
  prop = camelCaseClean(triple[1])

  text = subj+' '+prop+' '+obj+' .'
  textified_triples_noTypes_noTags.append(text)

# print(textified_triples_noTypes_noTags[0])
# print(len(textified_triples_noTypes_noTags))

In [76]:
import pickle

fineTuningDataset_path = '/content/' #Write path to save the dataset
with open(fineTuningDataset_path + 'triples_train_1_textified.txt', 'wb') as fh:
   pickle.dump(textified_triples_noTypes_noTags, fh)

In [77]:
# Free up memory
import gc
del dataset
del intermediate_dataset
del intermediate_dataset_temp
del sentences
del text_triples
del textified_triples_noTypes_noTags
del triples
del webnlg
gc.collect()

15

### FT-1.4 Format dataset for Sentence Transformers: CELL TO EDIT in experiments!

Load the files produced in the previous step

In [78]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

!pip install -U sentence-transformers==2.2.2

clear_output()

import pickle

# Load created dataset, with triples and scores
fineTuningDataset_path = '/content/' #Write path to save the dataset
pickle_off = open(fineTuningDataset_path + 'intermediate_dataset.txt', 'rb')
interm_dataset = pickle.load(pickle_off)

# Load textified triples
pickle_off = open(fineTuningDataset_path + 'triples_train_1_textified.txt', 'rb')
textified_triples = pickle.load(pickle_off)

In [79]:
# Print some examples
sample_triples = ['Aarhus_Airport | cityServed | "Aarhus, Denmark"', 'Aarhus_Airport | cityServed | Aarhus', 'Christian_Burns | genre | House_music', 'Athens_International_Airport | runwayLength | 3800.0', 'New_York_City | country | United_States']

for triple in sample_triples:
  print(triple)
  print(list(interm_dataset[1][triple])[:10])
  print(list(interm_dataset[0][triple])[:10])
  print()

print(len(textified_triples))
print(textified_triples[0])

Aarhus_Airport | cityServed | "Aarhus, Denmark"
['Aarhus Airport serves the city of Aarhus, Denmark.', 'The Aarhus is the airport of Aarhus, Denmark.']
['" A Wizard of Mars was Published in the United States ".', '"1928 SJ" is the former name of 1101 Clematis.', '"1999 SN5" was the former name of (66063) 1998 RO1.', '"87000823" is Asilomar Conference Grounds\' reference number in the National Register of Historic Places.', '"AIP Adv." is the abbreviation of AIP Advances.', '"Abh.Math.Semin.Univ.Hambg" is the abbreviation for Abhandlungen aus dem Mathematischen Seminar der Universität Hamburg.', '"Drake Stevens" is known as the comic character Airman.', '"Maka-Diyos, Maka-Tao, Makakalikasan at Makabansa" is a motto of the Philippines.', '"Mark Desmond" is also known as the comic character Blockbuster.', '"Mayor" is the title of the leader in Atlantic City, New Jersey.']

Aarhus_Airport | cityServed | Aarhus
['Aarhus airport serves the city of Aarhus.']
['" A Wizard of Mars was Published

The final dataset is a list of objects with two attributes, "texts", which is a list of two sentences (texts=[sentence1, sentence2]), and "label", which is the similarity score between these two sentences (label=score).

In [80]:
from tqdm import tqdm
from sentence_transformers import InputExample

triples = list(interm_dataset[0].keys())
# triples[999], textified_triples[999]

# num_negative_samples = 0
# num_positive_samples = 0

# Build a list that contains the number of samples for each category
# E.g. in our example: [23252141, 7645] = 23252141 negative samples and 7645 positive samples
num_samples = []

ft_sentencePair_score_list = []
#visualize_examples = []
for i, category in enumerate(interm_dataset):
    # We assume that the position of a data split correlates with its level of similarity between triples and text, so we get the score from that position
    # score must be between 0 and 1, see https://www.sbert.net/examples/training/sts/README.html, which is why we normalise it below
    score = float(i/(len(interm_dataset)-1))
    print('Category', i, end=': ')
    num_samples.append(0)
    for j, triple in enumerate(tqdm(triples)):
        for sentence in category[triple]:
            sentence1 = textified_triples[j]
            sentence2 = sentence
            inp_example = InputExample(texts=[sentence1, sentence2], label=score)
            #visualize_example = [[sentence1, sentence2], score]
            ft_sentencePair_score_list.append(inp_example)
            #visualize_examples.append(visualize_example)
            num_samples[i] += 1
            # if i == 0:
            #     num_negative_samples += 1
            # elif i == 1:
            #     num_positive_samples += 1

Category 0: 

100%|██████████| 3107/3107 [01:19<00:00, 38.94it/s]


Category 1: 

100%|██████████| 3107/3107 [00:00<00:00, 193059.40it/s]


In [81]:
# print(len(ft_sentencePair_score_list) == num_negative_samples + num_positive_samples)
print(len(ft_sentencePair_score_list) == sum(num_samples))
print(ft_sentencePair_score_list[0])
print(num_samples) # 2 splits (no overlap and full overlap): [23252141, 7645]

True
<InputExample> label: 0.0, texts: Aarhus Airport city served Aarhus, Denmark .; " A Wizard of Mars was Published in the United States ".
[23252141, 7645]


Balance the dataset so as to have as many examples of positive and negative data.

In [82]:
# CELL TO EDIT!
import random

# category0 = ft_sentencePair_score_list[:num_negative_samples] # [:23252141]
# category1 = ft_sentencePair_score_list[num_negative_samples:]

# Get the smallest data split
largest_split = 0
for num_sample in num_samples:
    if num_sample > largest_split:
        largest_split = num_sample
smallest_split = largest_split
for num_sample in num_samples:
    if num_sample < smallest_split:
       smallest_split = num_sample

# Get the position of the last data point of each split
ids_last_element_of_splits = [num_samples[0]]
separator = 1
while separator < len(num_samples):
    # Sum the previous and the new separator
    new_separator = ids_last_element_of_splits[-1]+num_samples[separator]
    ids_last_element_of_splits.append(new_separator)
    separator += 1

############################################################# EDIT BELOW #############################################################
category0 = ft_sentencePair_score_list[:ids_last_element_of_splits[0]] # [:23252141]
category1 = ft_sentencePair_score_list[ids_last_element_of_splits[0]:ids_last_element_of_splits[1]]

random.seed(42)
shuffled_category0 = random.sample(category0, len(category0))
shuffled_category1 = random.sample(category1, len(category1))

category0 = shuffled_category0[:smallest_split]
category1 = shuffled_category1[:smallest_split]

ft_sentencePair_score_list_new = category0 + category1
############################################################# EDIT ABOVE #############################################################

# print(category0[0].texts)
# print(ft_sentencePair_score_list[0].texts)
# print(ft_sentencePair_score_list[0].label)
# print(ft_sentencePair_score_list[1000].texts)
# print(ft_sentencePair_score_list[1000].label)

In [83]:
print(len(ft_sentencePair_score_list_new))
print(ids_last_element_of_splits)
# x = 0
# while x < 100:
#   print(str(ft_sentencePair_score_list_new[x].label)+': '+str(ft_sentencePair_score_list_new[x].texts))
#   x += 1

# print(largest_split)
# print(smallest_split)
# print(ids_last_element_of_splits)
# print(num_samples[0])
# print(num_samples[0]+num_samples[1])
# print(num_samples[0]+num_samples[1]+num_samples[2])
# print(num_samples[0]+num_samples[1]+num_samples[2]+num_samples[3])

15290
[23252141, 23259786]


In [84]:
import pickle
with open(fineTuningDataset_path + 'fine_tuning_dataset.txt', 'wb') as fh:
   pickle.dump(ft_sentencePair_score_list_new, fh)

#### Free up memory and Load created dataset if you want to have a look at it

In [85]:
import pickle

fineTuningDataset_path = '/content/'
pickle_off = open(fineTuningDataset_path + 'fine_tuning_dataset.txt', 'rb')
new_dataset = pickle.load(pickle_off)

In [86]:
# Should print the last datapoints of a split and the first of the next split (different label)
# print(new_dataset[smallest_split-1].texts)
# print(new_dataset[smallest_split-1].label)
# print(new_dataset[smallest_split].texts)
# print(new_dataset[smallest_split].label)

x = 0
while x < len(new_dataset):
  datapoint = new_dataset[x]
  print(datapoint.texts)
  print(datapoint.label)
  x += smallest_split
print()

['Acharya Institute of Technology number of postgraduate students 700 .', 'The Aston Martin V8 and Aston Martin RHAM/1 are related means of transportation.']
0.0
['Adams Township, Madison County, Indiana country United States .', 'Adams Township is in Madison County, in Indiana, in the United States.']
1.0



In [87]:
# Free up memory (needed for the whole notebook to run)
import gc
del category0
del category1
del ft_sentencePair_score_list
del ft_sentencePair_score_list_new
del interm_dataset
del new_dataset
del pickle_off
del shuffled_category0
del textified_triples
del triples
gc.collect()

0

## **FT-2 Fine-tuning**

### FT-2.1 Set parameters, load model: CELL TO EDIT in experiments! (INPUT NEEDED: path to save fine-tuned model)

In [88]:
# CELL TO EDIT!
from torch.utils.data import DataLoader
import torch
import math
from sentence_transformers import SentenceTransformer, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import logging
from datetime import datetime
import os
import gzip
import csv

#### Just some code to print debug information to stdout
logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
#### /print debug information to stdout

#Check if dataset exsist. If not, download and extract  it
'''sts_dataset_path = 'datasets/stsbenchmark.tsv.gz'

if not os.path.exists(sts_dataset_path):
    util.http_get('https://sbert.net/datasets/stsbenchmark.tsv.gz', sts_dataset_path)
'''

############################################################# EDIT BELOW #############################################################
# Read the dataset
model_name = 'nli-distilroberta-base-v2'
# Set random seed
torch.manual_seed(42)
# 128 uses much more memory but doesn't seem to be faster
train_batch_size = 64
num_epochs = 4
model_save_path = '/content/drive/MyDrive/Colab-dump/Lab_Week10/MyModel-'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S") #finetuned model path and name
############################################################# EDIT ABOVE #############################################################

# Load a pre-trained sentence transformer model
model = SentenceTransformer(model_name)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


### FT-2.2 Load dataset and create splits: CELL TO EDIT in experiments!

In [89]:
import pickle

fineTuningDataset_path = '/content/'
pickle_off = open(fineTuningDataset_path + 'fine_tuning_dataset.txt', 'rb')
input_examples = pickle.load(pickle_off)

In [90]:
# CELL TO EDIT!
import random

############################################################# EDIT BELOW #############################################################
random.seed(42)
shuffled_input_examples = random.sample(input_examples, len(input_examples)) # shuffles the ordering of filenames (deterministic given the chosen seed)

split_1 = int(0.70 * len(shuffled_input_examples))
split_2 = int(0.85 * len(shuffled_input_examples))
############################################################# EDIT ABOVE #############################################################

train_samples = shuffled_input_examples[:split_1]
dev_samples = shuffled_input_examples[split_1:split_2]
test_samples = shuffled_input_examples[split_2:]

In [91]:
len(train_samples), len(dev_samples), len(test_samples)

(10703, 2293, 2294)

### FT-2.3 Create model (use GPU)

In [92]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

# Development set: Measure correlation between cosine score and gold labels
logging.info("Read STSbenchmark dev dataset")
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')

# Configure the training. We skip evaluation in this example
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1) #10% of train data for warm-up
logging.info("Warmup-steps: {}".format(warmup_steps))

In [93]:
# Train the model (Use GPU)
model.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          evaluation_steps=1000,
          warmup_steps=warmup_steps,
          output_path=model_save_path)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Iteration:   0%|          | 0/168 [00:00<?, ?it/s]

Iteration:   0%|          | 0/168 [00:00<?, ?it/s]

Iteration:   0%|          | 0/168 [00:00<?, ?it/s]

Iteration:   0%|          | 0/168 [00:00<?, ?it/s]

In [94]:
# Free up memory
import gc
del input_examples
del dev_samples
del model
del test_samples
del train_samples
del shuffled_input_examples
gc.collect()

260